In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [32]:
from sklearn.impute import SimpleImputer            #Handling Missing values
from sklearn.preprocessing import StandardScaler    #Handling the Feature Scaling
from sklearn.preprocessing import OrdinalEncoder    # Ordinal Encoding

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

In [62]:
from sklearn.linear_model import LinearRegression,Lasso, Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [47]:
data=pd.read_csv('data/gemstone.csv')
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [48]:
data.drop('id',axis=1,inplace=True)

In [49]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [50]:
X=data.drop('price',axis=1)
Y=data['price']

In [51]:
x_cat=X.select_dtypes(include=object).columns
x_num=X.select_dtypes(exclude=object).columns

In [52]:
cut_map=["Fair","Good","Very Good","Premium","Ideal"]
clarity_map=["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]
color_map=["D","E","F","G","H","I","J"]


In [53]:
num_pipelene=Pipeline(
    steps=[
    ('imputet',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

cat_pipelene=Pipeline(
    steps=[
    ('imputet',SimpleImputer(strategy='most_frequent')),
    ('ordinalEncoder',OrdinalEncoder(categories=[cut_map,color_map,clarity_map])),
    ('scaler',StandardScaler())
    ]
)

In [54]:
#TO combine he above applied functions we use this columnransformer
processor=ColumnTransformer([
    ('num_pipeline',num_pipelene,x_num),
    ('cat_pipeline',cat_pipelene,x_cat)
])

In [55]:
X_train, X_test, Y_train,Y_test=train_test_split(X,Y,test_size=0.30,random_state=40)

In [57]:
X_train=pd.DataFrame(processor.fit_transform(X_train),columns=processor.get_feature_names_out())
X_test=pd.DataFrame(processor.transform(X_test),columns=processor.get_feature_names_out())


In [61]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.454106,-0.296545,-0.637842,0.636818,0.663672,0.622410,0.871502,-0.317026,-0.650314
1,3.027661,-1.589990,1.965484,2.494366,2.433180,2.182754,-0.133146,0.915044,-1.315348
2,0.540612,1.458846,-1.158507,0.672887,0.645523,0.811984,-1.137795,0.915044,-1.315348
3,0.562238,-2.513880,1.444819,0.817163,0.790714,0.549496,-1.137795,-0.317026,-1.315348
4,-0.843485,-0.019378,-0.637842,-0.851023,-0.888051,-0.865022,0.871502,1.531079,1.344786


In [63]:
regressor=LinearRegression()

In [64]:
regressor.fit(X_train,Y_train)

LinearRegression()

In [65]:
regressor.intercept_

3966.0501029512675

In [66]:
regressor.coef_

array([ 6444.18494694,   -95.79974015,   -67.01577837, -1823.10963   ,
          -8.19306509,  -460.63495897,    74.53336566,  -464.42173169,
         650.12826018])

In [71]:

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [90]:
## Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

     #Make Predictions
    y_pred=model.predict(X_test)


    print(list(models.keys())[i])
   
    
    mae, rmse, r2_square=evaluate_model(Y_test,y_pred)

    trained_model_list.append(list(models.keys())[i])
    r2_list.append(r2_square)

    
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    
    print("*"*40)
    
   
    

LinearRegression
Model Training Performance
RMSE: 1017.1722564866152
MAE: 674.6638217377892
R2 score 93.63431403063537
****************************************
Lasso
Model Training Performance
RMSE: 1015.6056722113335
MAE: 675.3910275839802
R2 score 93.65390698366912
****************************************
Ridge
Model Training Performance
RMSE: 1017.1731135131265
MAE: 674.6868170797206
R2 score 93.63430330371297
****************************************
Elasticnet
Model Training Performance
RMSE: 1536.886461343751
MAE: 1065.571661050515
R2 score 85.46751887552743
****************************************


In [73]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']